In [6]:
!pip install sklearn progressbar2

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached progressbar2-3.51.4-py2.py3-none-any.whl (25 kB)
  Using cached scikit_learn-0.23.1-cp36-cp36m-manylinux1_x86_64.whl (6.8 MB)
  Using cached python_utils-2.4.0-py2.py3-none-any.whl (12 kB)
  Using cached joblib-0.16.0-py3-none-any.whl (300 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=ecb04c6bfe749e8a46b062f660bae6c4e48c17d310ebe19d662eaf5bc1e6ad7e
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random
import math
from sklearn.metrics import classification_report, confusion_matrix
import progressbar

In [8]:
GB = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
#The variable GB is the memory size you want to use.
config = [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*GB))]
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], config)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
np.set_printoptions(threshold=40*40)


In [9]:
def load_data(res, source_labels, dataset='train', size=1., verbosity = False, shuffle = True):
    data = np.zeros((0, res, res, res), dtype=int)
    labels = np.empty(0, int)
    for i, label in enumerate(source_labels):
        filename = './output/' + label + '/output-' + dataset + '-' + str(res) + '.h5'
        file = h5py.File(filename, 'r')
        tensor_len = len(file['tensor'])
        picked_ids = range(math.ceil(tensor_len * size) - 1)
        if verbosity:
            print(filename)
            print(picked_ids)
            print('Picking ' + str(math.ceil(tensor_len * size)) + ' from ' + str(len(file['tensor'])) )
        data = np.concatenate((data, [file['tensor'][index] for index in picked_ids]))
        labels = np.concatenate((labels, np.full(len(picked_ids), i, dtype=int)))
        file.close()
        gc.collect()
    if shuffle:
        if verbosity:
            print("Shuffling data")
        rng_state = np.random.get_state()
        np.random.shuffle(data)
        np.random.set_state(rng_state)
        np.random.shuffle(labels)
    return (data, labels)



In [ ]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(res, res, res)),
        keras.layers.Dense(res*2, activation='relu'),
        keras.layers.Dense(res/2, activation='sigmoid'),
        keras.layers.Dropout(.2, input_shape=(2,)),
        keras.layers.Dense(40, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes):
    return data
model_type = 'dense'
prepare_model(24, 20, 1)

In [17]:
def prepare_model(res, num_classes, verbosity = 0):
    model = keras.Sequential([
        keras.layers.Conv3D(
                res, 
                kernel_size=(6),
                strides=(2),
                activation='relu', 
                kernel_initializer='he_uniform', 
                data_format="channels_last",
                input_shape=(res, res, res, 1)
            ),        
        keras.layers.Conv3D(
                res, 
                kernel_size=(5),
                strides=(2),
                activation='relu', 
                kernel_initializer='he_uniform'
            ),        
        keras.layers.Conv3D(
                res, 
                kernel_size=(2),
                activation='relu', 
                kernel_initializer='he_uniform'
            ),        
   
        keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(res/2, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    learning_rate = 0.0001

    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    if (verbosity == 1):
        print(model.summary())
    return model
def prepare_data(data, num_classes, res):
    return data.reshape(len(data), res, res, res, 1)
model_type = 'conv3d'
prepare_model(24, 10, 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 10, 10, 10, 24)    5208      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 3, 3, 3, 24)       72024     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 2, 2, 2, 24)       4632      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 1, 1, 1, 24)       0         
_________________________________________________________________
flatten (Flatten)            (None, 24)                0         
_________________________________________________________________
dense (Dense)                (None, 12)                300       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [4]:
def train(model, training, training_labels, res, num_classes, verbosity = 0):
    log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    if verbosity == 1:
        print("Log file " + log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    validation_split = 0.1
    epochs = 10 * num_classes
    
#    model.fit(
#        keras.utils.to_categorical(training, num_clases), 
#        training_labels, 
#        epochs=epochs)
    
    model.fit(
        prepare_data(training, num_classes), 
        training_labels, 
        epochs=epochs,
        verbose=verbosity,
        callbacks=[tensorboard_callback],
        validation_split=validation_split
    )

In [10]:
#Confusion Matrix and Classification Report
def print_confusion_matrix(test, test_labels, base_labels):
    Y_pred = model.predict_generator(prepare_data(test, len(base_labels)), len(test))
    y_pred = np.argmax(Y_pred, axis=1)
    print('Confusion Matrix')
    print(confusion_matrix(test_labels, y_pred))
    print('Classification Report')
    print(classification_report(test_labels, y_pred, target_names=base_labels))

In [11]:
#base_labels = ['bathtub', 'bed', 'chair', 'desk', 'dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet']
all_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 
                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
                 'vase', 'wardrobe', 'xbox']


In [ ]:
subclases = [10, 20, 30, 40]
resolutions = [24]#, 32]#, 48]
num_classes = 2

for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        base_labels = [all_labels[i]] + [all_labels[j]]
        print(i, j, base_labels)
        for n, res in enumerate(resolutions):
            training, training_labels = load_data(res, base_labels, 'train', 1, False)
            model = prepare_model(res, num_classes, 0)
            model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'
            train(model, training, training_labels, res, num_classes, 0)
            model.save(model_name)
            test, test_labels = load_data(res, base_labels, 'test', 1.0, False)
            test_loss, test_acc = model.evaluate(
                prepare_data(test, num_classes),  
                test_labels, 
                verbose=2
            )
            print('\nTest accuracy for ' + all_labels[i] + '-' + all_labels[j] + ' classes width res ' + str(res) + ':', test_acc)
            print_confusion_matrix(test, test_labels, all_labels)
            gc.collect()          


In [12]:
res = 24
model_type = 'conv3d'
test, test_labels = load_data(res, all_labels, 'test', 1.0, False)


In [13]:
def real_index(prediction, i, j):
    return j if prediction else i

def prediction_for(i, j, data):
    model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
    model = keras.models.load_model(model_name)
    prediction = model.predict(prepare_data(data, 2, 24))
    return [real_index(p, i, j) for p in list(map(np.argmax, prediction))]


In [29]:
print(test_labels[0])
test[0].shape
to_test = np.expand_dims(test[0], axis=0)
#model.predict(to_test)

0.0


In [ ]:
prediction = [x[:] for x in [[-1] * 40] * 40]
for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        prediction[i][j] = prediction_for(i, j, to_test)
        print(prediction[i][j])


In [ ]:
i = 0
j = 5

model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
model = keras.models.load_model(model_name)
#print(model.summary())
prediction = model.predict(test)
prediction

In [14]:
np.set_printoptions(threshold=40*40*40)

def real_index(prediction, i, j):
    return j if prediction else i

def prediction_for(i, j, data):
    model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
    model = keras.models.load_model(model_name)
    #print(model.summary())
    prediction = model.predict(prepare_data(data, 2, 24))
    return [real_index(p, i, j) for p in list(map(np.argmax, prediction))]



In [15]:
tf.get_logger().setLevel('ERROR')

# Calculate test prediction for each labels pair

In [18]:
prediction = [x[:] for x in [[-1] * 40] * 40]
for i in range(len(all_labels)):
    bar2 = progressbar.ProgressBar(max_value=len(all_labels) - (i + 1))
    for j in range(i + 1, len(all_labels)):
        bar2.update(j - (i + 1))
        prediction[i][j] = prediction_for(i, j, test)


N/A% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

In [19]:
p = [None] * len(test)
bar = progressbar.ProgressBar(max_value=len(test))
for e in range(len(test)):
    bar.update(e) 
    p[e] = []
    for i in range(40):
        for j in range(40):
            if prediction[i][j] == -1:
                p[e].append(-1)
            else:
                p[e].append(prediction[i][j][e])

 98% (2398 of 2428) |################### | Elapsed Time: 0:00:02 ETA:   0:00:00

In [20]:
def most_frequent(List): 
    return max(set(List), key = List.count) 
count = 0
for index in range(len(test)):
    real = test_labels[index]
    predicted = most_frequent(list(filter(lambda a: a > -1, p[index])))
    if int(real) == predicted: 
        count+=1
count

1821

In [ ]:
print(all_labels[int(test_labels [0])])
p[0]

In [39]:
np.save('prediction-conv3d.txt', prediction)

In [ ]:
prediction2 = np.load('prediction-conv3d.txt.npy', allow_pickle=True)

In [ ]:
for i in range(len(all_labels)):
    for j in range(i + 1, len(all_labels)):
        model_name = 'models/' + model_type + '-' + all_labels[i] + '-' + all_labels[j] + '.h5'        
        model = keras.models.load_model(model_name)
        print(model.summary())


In [ ]:
model = keras.models.load_model('models/dense-20-24')


In [ ]:
num_classes = 20
res = 24
base_labels = all_labels[:num_classes]
test, test_labels = load_data(res, base_labels, 'test', 1.0, False)


In [ ]:
test_loss, test_acc = model.evaluate(
            prepare_data(test, num_classes),  
            test_labels, 
            verbose=2
        )
print('\nTest accuracy for ' + str(num_classes) + ' classes width res ' + str(res) + ':', test_acc)
print_confusion_matrix(test, test_labels, base_labels)

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()

In [ ]:
if not len(test):
    test, test_labels = load_data(dim, base_labels, 'test', 100)
test_loss, test_acc = model.evaluate(test,  test_labels, verbose=2)
p = model.predict(test)
print('\nTest accuracy:', test_acc)

In [ ]:
index = random.randint(0, len(test))

predicted_index = np.argmax(p[index])
expected = base_labels[int(test_labels[index])]
predicted = base_labels[predicted_index]
draw(test[index], expected, predicted)
index

In [ ]:
num_classes = 3
base_labels = all_labels[:num_classes]
num_classes = num_classes
res = 24
training, training_labels = load_data(res, base_labels, 'train', 1, True)


In [ ]:
print(len(training))
model = prepare_model(res, num_classes, 1)
train(model, training, training_labels, res, num_classes, 0)


In [ ]:
print(base_labels)
test, test_labels = load_data(res, base_labels, 'test', 1.0, True)
print(len(test))
test_loss, test_acc = model.evaluate(
    test,  
    test_labels, 
    verbose=2
)


In [ ]:
p = model.predict(test)
for index in range(len(p)):
    predicted_index = np.argmax(p[index])
    expected = base_labels[int(test_labels[index])]
    predicted = base_labels[predicted_index]
    if expected != predicted:
        draw(test[index], expected, predicted)
